In [8]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import pandas as pd 
import pyttsx3
import threading

In [9]:
# --- SETUP ---
engine = pyttsx3.init()
engine.setProperty('rate', 150)
engine.setProperty('volume', 1.0)
model = joblib.load("../models/gesture_classifier_normalized.pkl")

In [10]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

In [11]:
def normalize_landmarks(landmarks):
    origin_x = landmarks[0].x
    origin_y = landmarks[0].y
    origin_z = landmarks[0].z

    normalized = []
    for lm in landmarks:
        normalized.extend([
            lm.x - origin_x,
            lm.y - origin_y,
            lm.z - origin_z
        ])
    return normalized

In [12]:
def speak(text):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    engine.say(text)
    engine.runAndWait()

In [13]:
# --- TRACKING VARIABLES ---
last_prediction = ""
prediction_count = 0
stable_prediction = ""

In [14]:
# --- CAMERA LOOP ---
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image = cv2.flip(frame, 1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            features = normalize_landmarks(hand_landmarks.landmark)

            if len(features) == 63:
                features_df = pd.DataFrame([features], columns=model.feature_names_in_)
                prediction = model.predict(features_df)[0]
                confidence = max(model.predict_proba(features_df)[0])

                print(f"Predicted: {prediction} | Confidence: {confidence:.2f}")

                # Show top 3 predictions for debugging
                probs = model.predict_proba(features_df)[0]
                top_indices = np.argsort(probs)[-3:][::-1]  # Top 3 indices

                print("Top 3 predictions:")
                for i in top_indices:
                    print(f"{model.classes_[i]}: {probs[i]:.2f}")

                # Prediction smoothing
                if prediction == last_prediction:
                    prediction_count += 1
                else:
                    prediction_count = 1
                    last_prediction = prediction

                if prediction_count >= 5:
                    if prediction != stable_prediction:
                        stable_prediction = prediction
                        print(f"✅ Stable Prediction: {stable_prediction}")
                        threading.Thread(target=speak, args=(stable_prediction,), daemon=True).start()
                    prediction_count = 0

                # Display stable prediction
                cv2.putText(image, f"{stable_prediction} ({confidence:.2f})", (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Sign Language Translator", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Predicted: F | Confidence: 0.26
Top 3 predictions:
F: 0.26
W: 0.22
B: 0.13
Predicted: F | Confidence: 0.20
Top 3 predictions:
F: 0.20
B: 0.17
W: 0.15
Predicted: C | Confidence: 0.34
Top 3 predictions:
C: 0.34
O: 0.22
B: 0.13
Predicted: C | Confidence: 0.38
Top 3 predictions:
C: 0.38
B: 0.26
F: 0.15
Predicted: C | Confidence: 0.24
Top 3 predictions:
C: 0.24
F: 0.21
B: 0.17
Predicted: C | Confidence: 0.24
Top 3 predictions:
C: 0.24
F: 0.21
B: 0.15
Predicted: C | Confidence: 0.24
Top 3 predictions:
C: 0.24
F: 0.23
B: 0.16
✅ Stable Prediction: C
Predicted: C | Confidence: 0.26
Top 3 predictions:
C: 0.26
F: 0.18
B: 0.16
Predicted: C | Confidence: 0.21
Top 3 predictions:
C: 0.21
F: 0.21
B: 0.17
Predicted: C | Confidence: 0.27
Top 3 predictions:
C: 0.27
F: 0.18
B: 0.14
Predicted: C | Confidence: 0.25
Top 3 predictions:
C: 0.25
F: 0.23
B: 0.18
Predicted: C | Confidence: 0.27
Top 3 predictions:
C: 0.27
B: 0.19
F: 0.19
Predicted: C | Confidence: 0.28
Top 3 predictions:
C: 0.28
B: 0.21
F: 0.21
Pr

Exception in thread Thread-11 (speak):
Traceback (most recent call last):
  File "C:\Users\ANAND\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\ANAND\sign_language_translator\venv310\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\ANAND\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ANAND\AppData\Local\Temp\ipykernel_5280\3582840316.py", line 5, in speak
  File "C:\Users\ANAND\sign_language_translator\venv310\lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


Predicted: I | Confidence: 0.47
Top 3 predictions:
I: 0.47
Y: 0.28
M: 0.07
✅ Stable Prediction: I
Predicted: I | Confidence: 0.49
Top 3 predictions:
I: 0.49
Y: 0.26
M: 0.08
Predicted: I | Confidence: 0.49
Top 3 predictions:
I: 0.49
Y: 0.28
M: 0.07
Predicted: I | Confidence: 0.54
Top 3 predictions:
I: 0.54
Y: 0.24
M: 0.06
Predicted: I | Confidence: 0.48
Top 3 predictions:
I: 0.48
Y: 0.26
M: 0.08
Predicted: I | Confidence: 0.49
Top 3 predictions:
I: 0.49
Y: 0.29
M: 0.06
Predicted: I | Confidence: 0.47
Top 3 predictions:
I: 0.47
Y: 0.29
M: 0.07
Predicted: I | Confidence: 0.46
Top 3 predictions:
I: 0.46
Y: 0.28
E: 0.07
Predicted: I | Confidence: 0.46
Top 3 predictions:
I: 0.46
Y: 0.27
M: 0.08
Predicted: I | Confidence: 0.46
Top 3 predictions:
I: 0.46
Y: 0.29
M: 0.07
Predicted: I | Confidence: 0.50
Top 3 predictions:
I: 0.50
Y: 0.27
M: 0.07
Predicted: I | Confidence: 0.48
Top 3 predictions:
I: 0.48
Y: 0.28
M: 0.07
Predicted: I | Confidence: 0.46
Top 3 predictions:
I: 0.46
Y: 0.27
M: 0.09
Pr